<a href="https://colab.research.google.com/github/AndrewstheBuilder/ScratchNeuralNetworks/blob/main/Iris_Micrograd_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# Load the Iris Dataset
import numpy as np
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the dataset
iris = load_iris()
X = iris.data  # Features
y = iris.target  # Labels (0, 1, or 2)


# Standardize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [16]:
# Do preprocessing
print('Features of dataset:',X[0])
print('Labels of dataset:',y)
X.shape

Features of dataset: [-0.90068117  1.01900435 -1.34022653 -1.3154443 ]
Labels of dataset: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


(150, 4)

In [1]:
pip install micrograd_andrews

In [18]:
import math

def softmax(logits):
    exps = [logit.exp() for logit in logits]
    sum_exps = sum(exps)
    return [exp_i / sum_exps for exp_i in exps]

def cross_entropy(probs, label):
    # Convert label to one-hot encoding
    one_hot = [0] * len(probs)
    one_hot[label] = 1
    # Compute cross-entropy loss
    loss = -sum((p_i + 1e-9).log() * y_i for p_i, y_i in zip(probs, one_hot))
    return loss

In [42]:
# Use Micrograd MLP to solve the Iris dataset.
from micrograd_andrews.engine import Value
from micrograd_andrews.nn import Neuron, Layer, MLP

model = MLP(X.shape[0], [2,3])
# Begin gradient descent iterations
parameters_data_log = []
parameters_grad_log = []
# Hyperparameters
learning_rate = 0.05
epochs = 50
for epoch in range(epochs):
  correct = 0
  parameters_data_log.append([])
  parameters_grad_log.append([])
  total_loss = 0.0
  for i in range(len(X_train)):
    logits = model(X_train[i])
    # print(logits)
    probs = softmax(logits)
    y_true = y_train[i]
    loss = cross_entropy(probs, y_true)
    total_loss += loss.data
    # print('actual loss:',loss.data)

    # Prediction
    pred_label = probs.index(max(probs, key=lambda p: p.data))
    if pred_label == y_true:
      correct += 1

    # Back propagation
    model.zero_grad()
    loss.backward()

    # Update parameters
    for p in model.parameters():
      parameters_data_log[epoch].append(p.data)
      parameters_grad_log[epoch].append(p.grad)
      p.data -= learning_rate * p.grad

  accuracy = correct / len(X_train) * 100
  avg_loss = total_loss / len(X_train)
  print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

Epoch 1/50, Loss: 0.9142, Accuracy: 60.00%
Epoch 2/50, Loss: 0.6492, Accuracy: 79.17%
Epoch 3/50, Loss: 0.5098, Accuracy: 81.67%
Epoch 4/50, Loss: 0.4270, Accuracy: 85.00%
Epoch 5/50, Loss: 0.3630, Accuracy: 86.67%
Epoch 6/50, Loss: 0.3087, Accuracy: 89.17%
Epoch 7/50, Loss: 0.2679, Accuracy: 93.33%
Epoch 8/50, Loss: 0.2347, Accuracy: 95.00%
Epoch 9/50, Loss: 0.2084, Accuracy: 95.83%
Epoch 10/50, Loss: 0.1881, Accuracy: 95.83%
Epoch 11/50, Loss: 0.1727, Accuracy: 96.67%
Epoch 12/50, Loss: 0.1606, Accuracy: 96.67%
Epoch 13/50, Loss: 0.1511, Accuracy: 96.67%
Epoch 14/50, Loss: 0.1437, Accuracy: 95.83%
Epoch 15/50, Loss: 0.1372, Accuracy: 95.83%
Epoch 16/50, Loss: 0.1316, Accuracy: 95.83%
Epoch 17/50, Loss: 0.1260, Accuracy: 95.83%
Epoch 18/50, Loss: 0.1211, Accuracy: 95.83%
Epoch 19/50, Loss: 0.1154, Accuracy: 95.83%
Epoch 20/50, Loss: 0.1111, Accuracy: 95.83%
Epoch 21/50, Loss: 0.1070, Accuracy: 95.83%
Epoch 22/50, Loss: 0.1033, Accuracy: 96.67%
Epoch 23/50, Loss: 0.1000, Accuracy: 96.6

In [38]:
print('mean of data:',np.mean(parameters_data_log, axis=1))
print('mean of gradients:',np.mean(parameters_grad_log, axis=1))
# np.array(parameters_grad_log).shape
# np.array(X_train).shape
np.mean(X_train, axis=1)

mean of data: [-0.01539417 -0.01420393 -0.01227521 -0.00939283 -0.00859827 -0.00905091
 -0.00953515 -0.00984313 -0.01003877 -0.010176   -0.01025027 -0.01031221
 -0.01036213 -0.01040266 -0.01041716 -0.0104329  -0.01044461 -0.01047293
 -0.01050355 -0.01053295 -0.01056191 -0.0105904  -0.01061821 -0.01064515
 -0.01066955 -0.01069434 -0.01071737 -0.01073871 -0.01075019 -0.01076631
 -0.0107823  -0.01079722 -0.01081083 -0.01082307 -0.01083395 -0.0108435
 -0.01085173 -0.01085868 -0.01086438 -0.01086885 -0.01087214 -0.01087428
 -0.01087531 -0.01087527 -0.01087419 -0.01087212 -0.01086909 -0.01086515
 -0.01086034 -0.01085338]
mean of gradients: [-3.14051216e-04 -1.91944797e-04 -5.57054037e-04 -3.76698499e-04
  3.99633222e-05  9.15690047e-05  7.68433242e-05  5.80188461e-05
  3.95525287e-05  3.04725239e-05  1.71785614e-05  1.37203783e-05
  1.08008000e-05  9.51346255e-06  6.91381697e-06  6.45727158e-06
  4.97921455e-06  6.22610900e-06  5.93084579e-06  5.85492958e-06
  5.78929621e-06  5.68952391e-06 

array([-0.78508498,  0.14538307,  0.44131506, -0.75434329, -1.03323761,
        0.22464841,  0.45510575, -0.58983559, -0.60707969, -0.27744848,
        0.20249631,  0.48194431,  0.51685118, -0.26165751, -0.41415327,
       -0.55613057,  0.27987316,  0.63837427,  0.3534739 ,  1.48406843,
       -0.03802976,  1.19935575,  0.09767163, -0.63433691,  1.09585064,
       -0.23576967, -0.66159413, -0.64593019, -0.32482646, -0.96620589,
        0.29262939, -0.71171522, -0.92395612, -0.60808308, -0.18342011,
       -0.67767674, -0.01338861,  1.73715865, -0.95844432,  0.12763944,
        0.07580465, -0.1103247 ,  0.36019171,  0.20249631, -0.47714105,
        0.03708769,  0.90076878, -0.47232756, -0.20482008, -0.82197703,
        0.41266683, -0.77972726, -0.86569517, -0.50901317,  0.56774615,
       -0.91090825,  0.42458573, -0.70796871, -0.50457798,  0.44499812,
       -0.6863067 ,  1.04034224,  0.16435611,  0.14232958,  1.22441702,
       -0.35722917, -1.19167583, -1.23995472, -0.17050725,  1.00

In [40]:
correct = 0

for i in range(len(X_test)):
    y_true = y_test[i]

    # Forward pass
    logits = model(X_test[i])
    probs = softmax(logits)

    # Prediction
    pred_label = probs.index(max(probs, key=lambda p: p.data))
    if pred_label == y_true:
        correct += 1

test_accuracy = correct / len(X_test) * 100
print(f"Test Accuracy: {test_accuracy:.2f}%")

Test Accuracy: 96.67%
